In [201]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

# Aplicativo Básico de Projeto - Nível 1

## **I-)** Resumir um artigo longo

#### **Processo**

 - **1.** Carregue o documento.

In [2]:
from langchain_openai import OpenAI
LLM = OpenAI() ### Criando instância para LLM


In [3]:
# Usar um caminho absoluto para abrir o arquivo
caminho_absoluto = ".\DATA\seja-bom-e-como-não-morrer.txt" # Troque - Diretório Correspondente

with open(caminho_absoluto, "r") as file:
    artigo = file.read()

In [4]:
print(type(artigo))

<class 'str'>


In [5]:
print(artigo[:270])

### Seja Bom

Abril de 2008

(Este ensaio é derivado de uma palestra na Startup School de 2008.)

Cerca de um mês após começarmos a Y Combinator, criamos a frase que se tornou nosso lema: Faça algo que as pessoas queiram. Aprendemos muito desde então, mas se eu tivesse 


 - **2.** Verifique a contagem de tokens.

In [6]:
num_tokens = LLM.get_num_tokens(artigo)
print(num_tokens)

3633


 - **3.** Divida em partes menores.

In [7]:
### Dividindo em pequenos chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  separators=["\n\n", "\n"],
  chunk_size=5000,
  chunk_overlap=350
)



In [8]:
article_chunks = text_splitter.create_documents([artigo])

In [9]:
print(f"Você tem {len(article_chunks)} chunks dentro de 1 artigo")

Você tem 3 chunks dentro de 1 artigo


 - **4.** Use uma cadeia LangChain predefinida para enviar as peças para ChatGPT e obter um resumo do documento.

In [10]:
from langchain.chains.summarize import load_summarize_chain

In [11]:
chain = load_summarize_chain(
    llm=LLM,
    chain_type="map_reduce"
)

In [12]:
article_summary= chain.run(article_chunks)

c:\Users\mathe\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [13]:
print(article_summary)



The essay "Seja Bom" explores the idea that creating something people want and focusing on benevolence rather than money can lead to successful businesses and startups. Examples from companies like Craigslist and Google support this concept. The author suggests that this idea could also be applied to nonprofits. Having loyal customers and maintaining their happiness is crucial for success, and benevolence can attract talented employees and aid in decision-making. However, other factors also play a role in achieving success.


## II-) Q&A (RAG) - Documentos

#### **Processo**

 - **1.** Carregue o documento de texto com um carregador de documentos.

In [14]:
from langchain_openai import OpenAI
LLM = OpenAI() ### Criando instância para LLM

In [15]:
from langchain.document_loaders import TextLoader

loader = TextLoader(".\DATA\seja-bom-e-como-não-morrer.txt") # Troque - Diretório Correspondente

In [16]:
document = loader.load()

In [17]:
print(type(document))

<class 'list'>


In [18]:
print(len(document))

1


In [19]:
print(document[0].metadata)

{'source': '.\\DATA\\seja-bom-e-como-não-morrer.txt'}


In [20]:
print(f"seu documento tem {len(document[0].page_content)} caracteres")

seu documento tem 13952 caracteres


 - **2.** Divida o documento em fragmentos com um divisor de texto.

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [22]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=400
)

In [23]:
document_chunks = text_splitter.split_documents(document)

In [24]:
print(f"Now you have {len(document_chunks)} chunks.")

Now you have 6 chunks.


- **3.** Converta os fragmentos em embeddings com OpenAIEmbeddings.

In [25]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

c:\Users\mathe\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


- **4.** Carregue os embeddings em um banco de dados de vetores FAISS.

In [26]:
from langchain.vectorstores import FAISS

In [27]:
stored_embeddings = FAISS.from_documents(document_chunks, embeddings)

- **5.** Crie uma cadeia RetrievalQA para recuperar os dados.

In [28]:
from langchain.chains import RetrievalQA

In [29]:
QA_chain = RetrievalQA.from_chain_type(
    llm=LLM,
    chain_type="stuff",
    retriever=stored_embeddings.as_retriever()
)

In [30]:
pergunta = """
Sobre o que é este artigo?
Descreva-o em menos de 100 palavras.
"""

In [31]:
QA_chain.run(pergunta)

'\n\nEste artigo discute a importância da benevolência em startups e como ela pode ser benéfica de várias maneiras, incluindo melhorar o moral, atrair ajuda de outras pessoas e ajudar na tomada de decisões. Ele também menciona exemplos de empresas de sucesso que seguiram essa abordagem, como o Google e o Craigslist. O autor sugere que a benevolência pode ser uma estratégia eficaz para startups e pode até mesmo ser aplicada em outros setores. '

## III-) Extraia dados estruturados de uma conversa

#### **Processo**

 - **1.** Use ResponseSchema para determinar quais dados queremos extrair.


In [39]:
from langchain.output_parsers import ResponseSchema

response_schemas = [
   ResponseSchema(
       name="cantor",
       description="nome do cantor"
   ),
   ResponseSchema(
       name="música",
       description="nome da música"
 )



]

 - **2.** Use StructuredOutputParser para arquivar os dados extraídos em um JSON dicionário.

In [40]:
from langchain.output_parsers import StructuredOutputParser

In [41]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [42]:
format_instructions = output_parser.get_format_instructions()

In [43]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"cantor": string  // nome do cantor
	"música": string  // nome da música
}
```


 - **3.** Crie o ChatPromptTemplate.

In [44]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import HumanMessagePromptTemplate

In [45]:
chat_model = ChatOpenAI()

In [46]:
template = """
Dado um comando do usuário,
extraia os nomes dos artistas e das músicas
{format_instructions}
{user_prompt}
"""

In [47]:
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)
    ],
    input_variables={"user_prompt"},
    partial_variables={"format_instructions": format_instructions}
)

 - **4.** Insira a mensagem do usuário.

In [48]:
user_message = prompt.format_prompt(
    user_prompt="Eu gosto da música New York, New York de Frank Sinatra"
)

In [49]:
user_chat_message = chat_model(user_message.to_messages())

c:\Users\mathe\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


- **5.** Extraia os dados e arquive-os no formato JSON.

In [50]:
extraction = output_parser.parse(user_chat_message.content)

In [52]:
print(extraction)

{'cantor': 'Frank Sinatra', 'música': 'New York, New York'}


In [53]:
print(type(extraction))

<class 'dict'>


## IV-) Avaliação de um aplicativo de perguntas e respostas

#### **Processo**

 - **1.** Carregue o documento de texto com um carregador de documentos.

In [55]:
from langchain import OpenAI

In [56]:
llm = OpenAI()

c:\Users\mathe\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [57]:
from langchain.document_loaders import TextLoader

In [61]:
loader = TextLoader("./DATA/seja-bom-e-como-não-morrer.txt")
document = loader.load()

In [62]:
print(f"esse documento tem {len(document[0].page_content)} caracteres")

esse documento tem 13952 caracteres


- **2.** Divida o documento em fragmentos com um divisor de texto.

In [63]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [64]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=400
)

In [65]:
document_chunks = text_splitter.split_documents(document)

In [66]:
print(f"Agora este documento tem {len(document_chunks)} chunks")

Agora este documento tem 6 chunks


- **3.** Converta os fragmentos em embeddings com OpenAIEmbeddings.

In [67]:
from langchain.embeddings.openai import OpenAIEmbeddings

In [68]:
embeddings = OpenAIEmbeddings()

- **4.** Carregue os embeddings em um banco de dados de vetores FAISS.

In [69]:
from langchain.vectorstores import FAISS

In [70]:
stored_embeddings = FAISS.from_documents(document_chunks, embeddings)

- **5.** Crie uma cadeia RetrievalQA para recuperar os dados, incluindo uma input_key para identificar o prompt do usuário **(a pergunta)**.

In [71]:
from langchain.chains import RetrievalQA

In [72]:
QA_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=stored_embeddings.as_retriever(),
    input_key="question"
)

- **6.** Crie um dicionário com as perguntas e respostas da avaliação.

In [84]:
question_and_answer = [
    {
        'question' : "Where is a whole neighborhood of YC-funded startups?", 
        'answer' :"In San Francisco"},
    {
        'question' : "What may be the most valuable  thing Paul Buchheit made for Google?", 
        'answer' : "The motto Don't be evil"}
]

- **7.** Use a cadeia RetrievalQA para avaliar manualmente o aplicativo.

In [85]:
predictions = QA_chain.apply(question_and_answer)

In [86]:
predictions

[{'question': 'Where is a whole neighborhood of YC-funded startups?',
  'answer': 'In San Francisco',
  'result': '\n\nThe whole neighborhood of YC-funded startups is located in Silicon Valley, specifically in the San Francisco Bay Area of California.'},
 {'question': 'What may be the most valuable  thing Paul Buchheit made for Google?',
  'answer': "The motto Don't be evil",
  'result': ' The idea of "Don\'t be evil."'}]

- **8.** Use uma cadeia QAEvalChain para que o aplicativo se avalie.

In [87]:
from langchain.evaluation.qa import QAEvalChain

In [88]:
evaluation_chain = QAEvalChain.from_llm(llm)

In [89]:
evaluate_responses = evaluation_chain.evaluate(
    question_and_answer,
    predictions,
    question_key="question",
    answer_key="answer"
)

In [90]:
evaluate_responses

[{'results': ' CORRECT'}, {'results': ' CORRECT'}]

## V-) Pergunte a um banco de dados

#### **Processo**

- **1.** Carregue o banco de dados.

In [92]:
!pip install langchain_experimental

  Obtaining dependency information for langchain_experimental from https://files.pythonhosted.org/packages/35/c4/7b29a25d1296834e28143df6cf8b0f1f10e18bbd2eefd849207a494bf86c/langchain_experimental-0.0.62-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-community<0.3.0,>=0.2.6 from https://files.pythonhosted.org/packages/3c/20/846d7fac8f6945a6f66b86cbef771f7697dfbb272d611a733cc9095329ce/langchain_community-0.2.7-py3-none-any.whl.metadata
  Obtaining dependency information for langchain<0.3.0,>=0.2.7 from https://files.pythonhosted.org/packages/8e/bf/b581a91c3238d93f1b57e093523c1120ee86fddf3a914e288701caba3620/langchain-0.2.7-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.3.0,>=0.2.10 from https://files.pythonhosted.org/packages/71/52/ea7b75d5363e215246873e3dda8098cea4eb80b4b327fe053bde7f7f4573/langchain_core-0.2.16-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-text-splitters<0.3.0,>=0.2.0 from http

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-astradb 0.1.0 requires langchain-core<0.2.0,>=0.1.31, but you have langchain-core 0.2.16 which is incompatible.
langchain-openai 0.0.8 requires langchain-core<0.2.0,>=0.1.27, but you have langchain-core 0.2.16 which is incompatible.
ragstack-ai 0.10.0 requires langchain==0.1.12, but you have langchain 0.2.7 which is incompatible.
ragstack-ai 0.10.0 requires langchain-community==0.0.28, but you have langchain-community 0.2.7 which is incompatible.
ragstack-ai 0.10.0 requires langchain-core==0.1.31, but you have langchain-core 0.2.16 which is incompatible.


In [91]:
from langchain_openai import OpenAI
llm = OpenAI()

In [93]:
from langchain import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [94]:
sqlite_db_path = "./DATA/street_tree_db.sqlite"

In [95]:
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

- **2.** Crie o SQLDatabaseChain.

In [96]:
db_chain = SQLDatabaseChain(
    llm=llm,
    database=db,
    verbose=True
)

c:\Users\mathe\anaconda3\Lib\site-packages\langchain_experimental\sql\base.py:77: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


- **3.** Faça perguntas em linguagem natural

In [98]:
db_chain.run("How many species of trees are in San Francisco?")



> Entering new SQLDatabaseChain chain...
How many species of trees are in San Francisco?
SQLQuery:SELECT COUNT(DISTINCT qSpecies) FROM street_trees
SQLResult: [(148,)]
Answer:148
> Finished chain.


'148'

## V-) Pergunte a um repositório do Github

#### **Processo**

- **1.** Carregue o repositório Github como uma coleção de documentos de texto.

In [99]:
root_dir = "data/thefuzz-master"

In [100]:
document_chunks = []

In [101]:
from langchain.document_loaders import TextLoader

In [102]:
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try:
            loader = TextLoader(
                os.path.join(dirpath, file),
                encoding="utf-8"
            )
            document_chunks.extend(loader.load_and_split())
        except Exception as e:
            pass

In [103]:
print(f"Nós temos {len(document_chunks)} chunks.")

Nós temos 11 chunks.


In [104]:
print(document_chunks[0].page_content[:300])

#!/usr/bin/env python

from rapidfuzz.fuzz import (
    ratio as _ratio,
    partial_ratio as _partial_ratio,
    token_set_ratio as _token_set_ratio,
    token_sort_ratio as _token_sort_ratio,
    partial_token_set_ratio as _partial_token_set_ratio,
    partial_token_sort_ratio as _partial_token_so


- **2.** Converta os documentos em embeddings com OpenAIEmbeddings.

In [105]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

In [106]:
embeddings = OpenAIEmbeddings()

- **3.** Carregue os embeddings em um banco de dados vetorial FAISS.

In [107]:
stored_embeddings = FAISS.from_documents(document_chunks, embeddings)

- **4.** Crie uma cadeia RetrievalQA para recuperar os dados.

In [109]:
from langchain_openai import ChatOpenAI

In [110]:
chat_model = ChatOpenAI()

In [111]:
from langchain.chains import RetrievalQA

In [112]:
qa_chain = RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="stuff",
    retriever=stored_embeddings.as_retriever()
)

In [113]:
question = """
What function do I use if I want to find 
the most similar item in a list of items?
"""

In [114]:
answer = qa_chain.run(question)

In [115]:
print(answer)

You can use the `extractWithoutOrder` function from the context provided to find the most similar item in a list of items. This function is designed to select the best match in a list of choices by comparing each choice to a query and returning the best match along with its score.


## VI-) Pergunte a uma API

#### **Processo**

**1.** Defina a API: url base e endpoints.

In [117]:
api_docs = """
BASE URL: https://restcountries.com/

API Documentation:

The API endpoint /v3.1/name/{name} Used to find informatin about 
a country. All URL parameters are listed below:
    - name: Name of country - Example: Italy, France
    
The API endpoint /v3.1/currency/{currency} Used to find information 
about a region. All URL parameters are listed below:
    - currency: 3 letter currency. Example: USD, COP

The API endpoint /v3.1/lang/{language} Used to find information 
about the official language of the country. All URL parameters 
are listed below:
    - language: language of the country. Example: English, Spanish
    
"""

**2.** Crie um APIChain com a API e o LLM.

In [118]:
from langchain.chains import APIChain

In [119]:
api_chain = APIChain.from_llm_and_api_docs(
    llm=llm,
    api_docs=api_docs,
    verbose=True,
    limit_to_domains=["https://restcountries.com/"]
)

**3.** Pergunte à API usando linguagem natural.

In [120]:
question = "Give me information about France in less than 100 words."

In [121]:
api_chain.run(question)



> Entering new APIChain chain...
 https://restcountries.com/v3.1/name/France?fields=name,capital,population,region,subregion,borders,languages,currencies
[{"name":{"common":"France","official":"French Republic","nativeName":{"fra":{"official":"République française","common":"France"}}},"currencies":{"EUR":{"name":"Euro","symbol":"€"}},"capital":["Paris"],"region":"Europe","subregion":"Western Europe","languages":{"fra":"French"},"borders":["AND","BEL","DEU","ITA","LUX","MCO","ESP","CHE"],"population":67391582}]

> Finished chain.


' France is a country in Western Europe with a population of 67.3 million. Its official currency is the Euro and its capital is Paris. It shares borders with several other countries, including Andorra, Belgium, Germany, Italy, Luxembourg, Monaco, Spain, and Switzerland. The official language of France is French.'

In [122]:
question2 = """
List the top 3 biggest countries 
where the official language is French.
"""

In [123]:
api_chain.run(question2)



> Entering new APIChain chain...
 
https://restcountries.com/v3.1/lang/French?fields=name,capital,area&sort=area&order=desc&per_page=3
[{"name":{"common":"Djibouti","official":"Republic of Djibouti","nativeName":{"ara":{"official":"جمهورية جيبوتي","common":"جيبوتي‎"},"fra":{"official":"République de Djibouti","common":"Djibouti"}}},"capital":["Djibouti"],"area":23200.0},{"name":{"common":"Wallis and Futuna","official":"Territory of the Wallis and Futuna Islands","nativeName":{"fra":{"official":"Territoire des îles Wallis et Futuna","common":"Wallis et Futuna"}}},"capital":["Mata-Utu"],"area":142.0},{"name":{"common":"Luxembourg","official":"Grand Duchy of Luxembourg","nativeName":{"deu":{"official":"Großherzogtum Luxemburg","common":"Luxemburg"},"fra":{"official":"Grand-Duché de Luxembourg","common":"Luxembourg"},"ltz":{"official":"Groussherzogtum Lëtzebuerg","common":"Lëtzebuerg"}}},"capital":["Luxembourg"],"area":2586.0},{"name":{"common":"Mali","official":"Republic of Mali","nativ

' The top 3 biggest countries where the official language is French are Djibouti, Wallis and Futuna, and Luxembourg.'

## VII-) Chatbot com Personalidade e Memória - (corrigir)

#### **Processo**

- **1.** Defina a personalidade do chatbot.

In [125]:
from langchain.prompts.prompt import PromptTemplate

In [124]:
chatbot_role = """
You are Master Yoda, a warrior and a monk.
Your goal is to help the user to strengthen her performance and spirit.

{chat_history}
Human: {human_input}
Chatbot:
"""

 - **2.** Incluir a personalidade no modelo de prompt.

In [127]:
prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"],
    template=chatbot_role
)

 - **3.** Configure a memória do chatbot.

In [128]:
from langchain.memory import ConversationBufferMemory

In [129]:
chatbot_memory = ConversationBufferMemory(
    memory_key="chat_history"
)

 - **4.** Crie o chatbot usando uma cadeia com o LLM, o prompt e a memória.


In [141]:
from langchain_openai import OpenAI
from langchain import LLMChain

In [142]:
llm = OpenAI()

In [143]:
yoda_chatbot = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=chatbot_memory
)

 - **5.** Faça perguntas para verificar sua personalidade e memória

In [144]:
question = "Master Yoda, how should I have to face my day?"

In [145]:
yoda_chatbot.predict(human_input=question)



> Entering new LLMChain chain...
Prompt after formatting:

You are Master Yoda, a warrior and a monk.
Your goal is to help the user to strengthen her performance and spirit.

Human: Master Yoda, how should I have to face my day?
AI:  In facing your day, focus on the present moment and let go of any worries or fears. Remember the power of positivity and approach each task with a clear mind and a calm spirit. Trust in your abilities and have faith in the Force. Stay mindful and stay strong, and you will succeed in facing your day.
Human: 
Master Yoda,
How can I deal with an enemy that wants to kill me?

AI: 
First and foremost, always remember to never underestimate your enemy. Stay vigilant and alert at all times. Use your knowledge and skills to outsmart them, and do not let anger or fear cloud your judgement. Remember, violence is not always the answer. Seek a peaceful resolution, but be prepared to defend yourself if necessary. And always remember, the Force will guide and protect 

'In facing your day, focus on the present moment and let go of any worries or fears. Remember the power of positivity and approach each task with a clear mind and a calm spirit. Trust in your abilities and have faith in the Force. Stay mindful and stay strong, and you will succeed in facing your day.'

In [146]:
question2 = """
Master Yoda,
How can I deal with an enemy that wants to kill me?
"""

In [147]:
yoda_chatbot.predict(human_input=question2)



> Entering new LLMChain chain...
Prompt after formatting:

You are Master Yoda, a warrior and a monk.
Your goal is to help the user to strengthen her performance and spirit.

Human: Master Yoda, how should I have to face my day?
AI:  In facing your day, focus on the present moment and let go of any worries or fears. Remember the power of positivity and approach each task with a clear mind and a calm spirit. Trust in your abilities and have faith in the Force. Stay mindful and stay strong, and you will succeed in facing your day.
Human: 
Master Yoda,
How can I deal with an enemy that wants to kill me?

AI: 
First and foremost, always remember to never underestimate your enemy. Stay vigilant and alert at all times. Use your knowledge and skills to outsmart them, and do not let anger or fear cloud your judgement. Remember, violence is not always the answer. Seek a peaceful resolution, but be prepared to defend yourself if necessary. And always remember, the Force will guide and protect 

'First and foremost, always remember to never underestimate your enemy. Stay vigilant and alert at all times. Use your knowledge and skills to outsmart them, and do not let anger or fear cloud your judgement. Remember, violence is not always the answer. Seek a peaceful resolution, but be prepared to defend yourself if necessary. And always remember, the Force will guide and protect you. Trust in its strength and you will overcome any threat.'

In [148]:
question3="""
Master Yoda,
Do you remember what was my first question today?
"""

In [149]:
yoda_chatbot.predict(human_input=question3)



> Entering new LLMChain chain...
Prompt after formatting:

You are Master Yoda, a warrior and a monk.
Your goal is to help the user to strengthen her performance and spirit.

Human: Master Yoda, how should I have to face my day?
AI:  In facing your day, focus on the present moment and let go of any worries or fears. Remember the power of positivity and approach each task with a clear mind and a calm spirit. Trust in your abilities and have faith in the Force. Stay mindful and stay strong, and you will succeed in facing your day.
Human: 
Master Yoda,
How can I deal with an enemy that wants to kill me?

AI: 
First and foremost, always remember to never underestimate your enemy. Stay vigilant and alert at all times. Use your knowledge and skills to outsmart them, and do not let anger or fear cloud your judgement. Remember, violence is not always the answer. Seek a peaceful resolution, but be prepared to defend yourself if necessary. And always remember, the Force will guide and protect 

'As mentioned before, I am an AI and do not have the ability to remember past conversations. But I am here to help you in any way I can. Is there something else you would like to discuss or ask?'

## VIII-) RAG com DeepLake 

#### **Processo**

- **1.** Carregue as credenciais do DeepLake.

In [202]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [203]:
os.environ["ACTIVELOOP_TOKEN"] = os.environ["DEEPLAKE_API_KEY"]

In [204]:
my_activeloop_org_id = os.environ["ACTIVELOOP_ORG_ID"]

In [205]:
my_activeloop_dataset_name = "basic-rag-with-deeplake"

In [206]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA


- **2.** Crie o documento externo.

In [207]:
usa_curious_facts = [
    """
    The US celebrates Independence Day from the British Empire 
    on July 4. However, the country’s Declaration of Independence 
    was passed on July 2. It was only officially ratified on July 4.
    """,
    """
    The very first documented European to arrive in North America was 
    the Spaniard Juan Ponce de León, who landed in Florida in 1513.
    """
]

- **3.** Divida-o em pequenos fragmentos.

In [208]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 0
)
doc_chunks = text_splitter.create_documents(usa_curious_facts)
print(f"Este documento tem {len(doc_chunks)} chunks.")

Este documento tem 2 chunks.


- **4.** Crie o banco de dados vetorial com DeepLake.

In [212]:
!pip install "deeplake[enterprise]"

In [218]:
embeddings = OpenAIEmbeddings()


In [219]:
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

In [220]:
db = DeepLake(
    dataset_path=dataset_path,
    embedding=embeddings
)

ImportError: Could not import deeplake python package. Please install it with `pip install deeplake[enterprise]`.

- **5.** Carregue os embeddings.

- **6.**  Crie a cadeia de controle de qualidade.

- **7.**  Pergunte ao aplicativo sobre o documento.

- **8.**  Adicione novos dados ao documento.

- **9.** Atualize o banco de dados.

- **10.** Pergunte ao aplicativo sobre os novos dados